In [108]:
import pyodbc
import pandas as pd
import math

con = pyodbc.connect('DRIVER={SQL Server};SERVER=localhost;DATABASE=TASMC_ICU_Intensix')
cursor = con.cursor()

In [106]:
sql = "SELECT [PatientID],[Time] \
    ,MAX( CASE WHEN [ParameterID] in ('4704', \
                                       '6636', \
                                       '6637', \
                                       '7719', \
                                       '8417', \
                                       '8719', \
                                       '8720', \
                                       '9630', \
                                       '10997') THEN [Value] ELSE NULL END) HR \
    ,MAX( CASE WHEN [ParameterID] in ('4727', \
                                      '6627', \
                                      '6628', \
                                      '9231', \
                                      '9434', \
                                      '9474') THEN [Value]  ELSE NULL END) ArtBPS \
    ,MAX( CASE WHEN [ParameterID] in ('5433', \
                                      '9472', \
                                      '10825', \
                                      '10999', \
                                      '11084') THEN [Value]  ELSE NULL END) ArtBPM \
    ,MAX( CASE WHEN [ParameterID] in ('4721', \
                                      '5532', \
                                      '5644', \
                                      '5945', \
                                      '5952', \
                                      '6380', \
                                      '7104') THEN [Value]  ELSE NULL END) CVP \
    ,MAX( CASE WHEN [ParameterID] in ('4717','9649') THEN [Value] ELSE NULL END) PAPD \
    ,MAX( CASE WHEN [ParameterID] in ('4612','11017') THEN [Value]  ELSE NULL END ) RR_total \
    ,MAX( CASE WHEN [ParameterID] in ('7295','11016') THEN [Value]  ELSE NULL END ) RR_mandatory \
    ,MAX( CASE WHEN [ParameterID] in ('4631') THEN [Value]  ELSE NULL END ) Spo2 \
    ,MAX( CASE WHEN [ParameterID] in ('4960') THEN [Value]  ELSE NULL END ) ST1 \
    ,MAX( CASE WHEN [ParameterID] in ('4961') THEN [Value]  ELSE NULL END ) ST2 \
    ,MAX( CASE WHEN [ParameterID] in ('4962') THEN [Value]  ELSE NULL END ) ST3 \
    ,MAX( CASE WHEN [ParameterID] in ('4611') THEN [Value]  ELSE NULL END ) Fio2 \
    ,MAX( CASE WHEN [ParameterID] in ('6173') THEN [Value]  ELSE NULL END ) NiBPM \
    ,MAX( CASE WHEN [ParameterID] in ('6934') THEN [Value]  ELSE NULL END ) BIS \
    ,MAX( CASE WHEN [ParameterID] in ('4726', '8420') THEN [Value]  ELSE NULL END ) NiBPS \
    FROM  [TASMC_ICU_Intensix].[dbo].[Signals] \
    WHERE  [TASMC_ICU_Intensix].[dbo].[Signals].[Param_Type] = ' HR ArtBPS ArtBPM CVP PAPD RR_total RR_mandatory Spo2 ST1 ST2 ST3 Fio2'\
    GROUP BY [PatientID], [Time];" 
df=pd.read_sql(sql, con)   

In [107]:
df=df.sort_values(by=['PatientID','Time'], ascending=[True,True])
len(df)

126523

In [102]:
#Add column:
df['Tag'] = pd.Series(0, index=df.index)
df['Type'] = pd.Series('NULL', index=df.index)

In [103]:
#change units:
df.ix[:,'CVP']=df.ix[:,'CVP']/133
df.ix[:,'ArtBPM']=df.ix[:,'ArtBPM']/133
df.ix[:,'ArtBPS']=df.ix[:,'ArtBPS']/133
df.ix[:,'PAPD']=df.ix[:,'PAPD']/133
df.ix[:,'NiBPM']=df.ix[:,'NiBPM']/133

#df.ix[:,'NiBPM']=df.ix[:,'NiBPM']/133

In [13]:
for i in range(len(df.index)):
    Mtype=''
    Tag=0
    #Hypovolemia 1 or 2
    if df.ix[i,'CVP'] is not None and df.ix[i,'ArtBPM'] is not None:
        if not math.isnan(df.ix[i,'ArtBPM']) and not math.isnan(df.ix[i,'CVP']):
            if df.ix[i,'ArtBPM']<50 and df.ix[i,'CVP']<5 and df.ix[i,'CVP']>-10:
                Mtype =Mtype+', Hypovolemia 1 or 2'
                Tag=1
    #Hypovolemia 3
    if df.ix[i,'ArtBPM'] is not None:
        if df.ix[i,'ArtBPM']<50:
                Mtype =Mtype+', Hypovolemia 3'
                Tag=1
    #Hypovolemia 4
    if df.ix[i,'CVP'] is not None and df.ix[i,'NiBPM'] is not None:
        if not math.isnan(df.ix[i,'NiBPM']) and not math.isnan(df.ix[i,'CVP']):
            if df.ix[i,'NiBPM']<50 and df.ix[i,'CVP']<5 and df.ix[i,'CVP']>-10:
                Mtype =Mtype+', Hypovolemia 4'
                Tag=1
    #LV shock 1 
    if df.ix[i,'CVP'] is not None and df.ix[i,'PAPD'] is not None and df.ix[i,'ArtBPS'] is not None:
        if not math.isnan(df.ix[i,'PAPD']) and not math.isnan(df.ix[i,'CVP']) and not math.isnan(df.ix[i,'ArtBPS']):
            if df.ix[i,'ArtBPS']<78 and df.ix[i,'CVP']>16 and df.ix[i,'CVP']<35 and df.ix[i,'PAPD']>16 and df.ix[i,'PAPD']<60:
                Mtype = Mtype +', LV shock 1'
                Tag=1
    #LV shock 2
    if df.ix[i,'CVP'] is not None and df.ix[i,'ArtBPS'] is not None:
        if not math.isnan(df.ix[i,'ArtBPS']) and not math.isnan(df.ix[i,'CVP']):
            if df.ix[i,'ArtBPS']<78 and df.ix[i,'CVP']<35 and df.ix[i,'CVP']>16:
                Mtype =Mtype+', LV shock 2'
                Tag=1 
    #Agitation 1 
    if df.ix[i,'HR'] is not None and df.ix[i,'BIS'] is not None and df.ix[i,'ArtBPS'] is not None and df.ix[i,'NiBPS'] is not None:
        if not math.isnan(df.ix[i,'HR']) and not math.isnan(df.ix[i,'BIS']) and not math.isnan(df.ix[i,'ArtBPS']) and not math.isnan(df.ix[i,'NiBPS']):
            if df.ix[i,'BIS']>80 and df.ix[i,'HR']>110 and (df.ix[i,'ArtBPS']>140 | df.ix[i,'NiBPS']>140):
                Mtype = Mtype +', Agitation 1'
                Tag=1
    #Agitation 2 
    if df.ix[i,'RR_total'] is not None and df.ix[i,'RR_mandatory'] is not None and df.ix[i,'HR'] is not None and df.ix[i,'BIS'] is not None and df.ix[i,'ArtBPS'] is not None and df.ix[i,'NiBPS'] is not None:
        if df.ix[i,'RR_total'] is not None and not math.isnan(df.ix[i,'RR_mandatory']) and not math.isnan(df.ix[i,'HR']) and not math.isnan(df.ix[i,'BIS']) and not math.isnan(df.ix[i,'ArtBPS']) and not math.isnan(df.ix[i,'NiBPS']):
            if df.ix[i,'BIS']>80 and df.ix[i,'HR']>110 and (df.ix[i,'ArtBPS']>140 | df.ix[i,'NiBPS']>140) and (df.ix[i,'RR_mandatory']<df.ix[i,'RR_total'] | df.ix[i,'RR_total']>25):
                Mtype = Mtype +', Agitation 2'
                Tag=1
    #Bradycardia 1
    if df.ix[i,'HR'] is not None:
        if not math.isnan(df.ix[i,'HR']):
            if df.ix[i,'HR']<45:
                Mtype = Mtype +', Bradycardia 1' 
                Tag=1
    #Bradycardia Hypotension
    if df.ix[i,'HR'] is not None and df.ix[i,'ArtBPS'] is not None:
        if not math.isnan(df.ix[i,'HR']) and not math.isnan(df.ix[i,'ArtBPS']):
            if df.ix[i,'HR']<45 and df.ix[i,'ArtBPS']<78:
                Mtype = Mtype +', Bradycardia Hypotension' 
                Tag=1
    #Tachycardia Hypotension
    if df.ix[i,'HR'] is not None and df.ix[i,'ArtBPS'] is not None:
        if not math.isnan(df.ix[i,'HR']) and not math.isnan(df.ix[i,'ArtBPS']):
            if df.ix[i,'HR']>120 and df.ix[i,'ArtBPS']<78:
                Mtype = Mtype +', Tachycardia'
                Tag=1
    #Tachycardia
    if df.ix[i,'HR'] is not None:
        if not math.isnan(df.ix[i,'HR']):
            if df.ix[i,'HR']>120:
                Mtype = Mtype +', Tachycardia'
                Tag=1
    df.ix[i,'Type']=Mtype
    df.ix[i,'Tag']=Tag             


In [14]:
dtest=df[(df.ArtBPM<50) & (df.CVP<5) & (df.CVP>-10)]
len(dtest.index)

119

In [15]:
dtest=df[(df.ArtBPS<78) & (df.CVP<35) & (df.CVP>16)& (df.PAPD<60)&(df.PAPD>16)]
dtest

,PatientID,Time,HR,ArtBPS,ArtBPM,CVP,PAPD,RR_total,RR_mandatory,Spo2,ST1,ST2,ST3,Fio2,Tag,Type


In [16]:
dtest=df[(df.HR>120)]
len(dtest)

8140

In [17]:
dtest=df[(df.HR<45)]
len(dtest)

14

In [541]:
df.head()

,PatientID,Time,Alarm,HR,ArtBPS,ArtBPM,CVP,PAPD,RR_total,RR_mandatory,Spo2,ST1,ST2,ST3,Fio2,Tag,Type
1482,20069,2013-02-14 16:00:00,True,NaN,78.189101,NaN,NaN,0.587888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,
15419,20069,2013-02-14 16:00:00,None,119.0,NaN,60.145427,11.026662,NaN,17.0,17.0,92.5,0.019,0.018,-0.011,60.0,0,
1540,20069,2013-02-14 19:00:00,None,110.0,NaN,60.145427,10.024241,NaN,18.0,18.0,99.0,0.009,0.016,-0.001,60.0,0,
7940,20069,2013-02-14 19:00:00,True,NaN,79.191502,NaN,NaN,0.595425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,
880,20069,2013-02-15 01:00:00,None,NaN,NaN,53.128466,15.036361,NaN,18.0,18.0,99.0,-0.040,-0.007,-0.011,50.0,0,


In [436]:
matrix=df.iloc[:,2:15].copy()

,PatientID,Time,HR,ArtBPS,ArtBPM,CVP,PAPD,RR_total,RR_mandatory,Spo2,...,ArtBPM_AVG,CVP_AVG,PAPD_AVG,RR_total_AVG,RR_mandatory_AVG,Spo2_AVG,ST1_AVG,ST2_AVG,ST3_AVG,Fio2_AVG
14953,20069,2013-02-14 16:00:00,119.0,78.189101,60.145427,11.026662,0.587888,17.0,17.0,92.5,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
21290,20069,2013-02-14 19:00:00,110.0,79.191502,60.145427,10.024241,0.595425,18.0,18.0,99.0,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1385,20069,2013-02-15 01:00:00,159.0,71.172095,53.128466,15.036361,0.535129,18.0,18.0,99.0,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
11429,20069,2013-02-15 03:00:00,100.0,73.176941,53.128466,14.033932,0.550203,18.0,18.0,97.0,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
29980,20069,2013-02-15 05:00:00,111.0,98.237518,65.157550,14.033932,0.738628,18.0,18.0,98.0,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [437]:
matrix.head()

,HR,ArtBPS,ArtBPM,CVP,PAPD,RR_total,RR_mandatory,Spo2,ST1,ST2,ST3,Fio2,Tag
5492,119.0,78.189101,60.145427,11.026662,0.587888,17.0,17.0,92.5,0.019,0.018,-0.011,60.0,0
10815,110.0,79.191502,60.145427,10.024241,0.595425,18.0,18.0,99.0,0.009,0.016,-0.001,60.0,0
7204,159.0,71.172095,53.128466,15.036361,0.535129,18.0,18.0,99.0,-0.040,-0.007,-0.011,50.0,1
19564,100.0,73.176941,53.128466,14.033932,0.550203,18.0,18.0,97.0,-0.006,0.010,-0.003,40.0,0
15874,111.0,98.237518,65.157550,14.033932,0.738628,18.0,18.0,98.0,-0.020,-0.003,-0.002,40.0,0


In [438]:
normalized_data = (matrix - matrix.min()) / (matrix.max() - matrix.min())
normalized_data.describe()

,HR,ArtBPS,ArtBPM,CVP,PAPD,RR_total,RR_mandatory,Spo2,ST1,ST2,ST3,Fio2,Tag
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,0.462755,0.405789,0.297610,0.145238,0.405789,0.284251,0.663015,0.905059,0.977141,0.951635,0.870529,0.308233,0.173500
std,0.083148,0.077457,0.064063,0.038272,0.077457,0.081006,0.221732,0.104013,0.020901,0.020319,0.018918,0.268764,0.378688
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400901,0.352159,0.262136,0.132022,0.352159,0.214286,0.600000,0.883333,0.975925,0.950938,0.870200,0.000000,0.000000
50%,0.468468,0.401993,0.291262,0.140449,0.401993,0.261905,0.700000,0.933333,0.976551,0.951664,0.870838,0.333333,0.000000
75%,0.522523,0.451827,0.330097,0.146067,0.451827,0.309524,0.800000,0.966667,0.978696,0.953611,0.871663,0.500000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [439]:
from sklearn.cluster import KMeans

predictKM = KMeans(n_clusters=2)
predictKM.fit(normalized_data)
labels = predictKM.predict(normalized_data)

In [440]:
labels

array([1, 1, 0, ..., 1, 1, 0])

In [447]:
sum((labels!=df.Tag)/len(df))

0.99999999999989808

''

In [18]:
for i in range(2,len(df.index)):
    Type=''
    t1=df.ix[i-2,'Time'].to_pydatetime()
    t2=df.ix[i-1,'Time'].to_pydatetime()
    t3=df.ix[i,'Time'].to_pydatetime()
    MType=df.ix[i,'Type']
    
    
    if (t2-t1).total_seconds()==60.0 and (t3-t2).total_seconds()==60.0 and df.ix[i,'PatientID']==df.ix[i-1,'PatientID'] and df.ix[i-1,'PatientID']==df.ix[i-2,'PatientID']:
        #SVT & Hypotension
        if df.ix[i,'HR'] is not None and df.ix[i-1,'HR'] is not None and df.ix[i-2,'HR'] is not None and df.ix[i,'ArtBPS'] is not None and df.ix[i-1,'ArtBPS'] is not None and df.ix[i-2,'ArtBPS'] is not None:
            if not math.isnan(df.ix[i,'HR']) and not math.isnan(df.ix[i-1,'HR']) and not math.isnan(df.ix[i-2,'HR']) and not math.isnan(df.ix[i,'ArtBPS']) and not math.isnan(df.ix[i-1,'ArtBPS']) and not math.isnan(df.ix[i-2,'ArtBPS']):
                if ((df.ix[i,'ArtBPS']*1.25 <df.ix[i-1,'ArtBPS'])| (df.ix[i,'ArtBPS']*1.25<df.ix[i-2,'ArtBPS']) |(df.ix[i-1,'ArtBPS']*1.25<df.ix[i-2,'ArtBPS']))&((df.ix[i,'HR']>df.ix[i-1,'HR']*1.4)| (df.ix[i,'HR']>df.ix[i-2,'HR']*1.4) |(df.ix[i-1,'HR']>df.ix[i-2,'HR']*1.4)):
                    df.ix[i,'Tag']=1
                    df.ix[i-1,'Tag']=1
                    df.ix[i-2,'Tag']=1
                    if (df.ix[i,'ArtBPS']<50 | df.ix[i-1,'ArtBPS']<50 | df.ix[i-2,'ArtBPS']<50)
                        df.ix[i-2,Type]=MType+', SVT & Hypotension 2'
                    else
                        df.ix[i-2,Type]=MType+', SVT & Hypotension 1'
                    print("hi")
    if (((t2-t1).total_seconds()==120.0) | ((t3-t2).total_seconds()==120.0) | ((t3-t2).total_seconds()==180.0) | ((t2-t1).total_seconds()==180.0)) and df.ix[i,'PatientID']==df.ix[i-1,'PatientID'] and df.ix[i-1,'PatientID']==df.ix[i-2,'PatientID']:
        if df.ix[i,'HR'] is not None and df.ix[i-1,'HR'] is not None and df.ix[i-2,'HR'] is not None and df.ix[i,'ArtBPS'] is not None and df.ix[i-1,'ArtBPS'] is not None and df.ix[i-2,'ArtBPS'] is not None:
            if not math.isnan(df.ix[i,'HR']) and not math.isnan(df.ix[i-1,'HR']) and not math.isnan(df.ix[i-2,'HR']) and not math.isnan(df.ix[i,'ArtBPS']) and not math.isnan(df.ix[i-1,'ArtBPS']) and not math.isnan(df.ix[i-2,'ArtBPS']):
                if ((df.ix[i,'ArtBPM']*1.25 <df.ix[i-1,'ArtBPM'])| (df.ix[i,'ArtBPM']*1.25<df.ix[i-2,'ArtBPM']) |(df.ix[i-1,'ArtBPM']*1.25<df.ix[i-2,'ArtBPM']))&((df.ix[i,'HR']>df.ix[i-1,'HR']*1.4)| (df.ix[i,'HR']>df.ix[i-2,'HR']*1.4) |(df.ix[i-1,'HR']>df.ix[i-2,'HR']*1.4)):
                    df.ix[i,'Tag']=1
                    df.ix[i-1,'Tag']=1
                    df.ix[i-2,'Tag']=1
                    df.ix[i-2,Type]=MType+' ,SVT & Hypotension'
                    print("hi")

In [19]:
#Hypovolemia1 - 30
#LV shock 1 - 0 
#Bradycardia1 - 6
#Tachycardia - 3433
#SVT & Hypotension - 0
len(df[df.Type == ' SVT & Hypotension'])

0

In [59]:
data=df.iloc[:,2:14].copy()

,HR,ArtBPS,ArtBPM,CVP,PAPD,RR_total,RR_mandatory,Spo2,ST1,ST2,ST3,Fio2
14953,119.0,78.189101,60.145427,11.026662,0.587888,17.0,17.0,92.5,0.019,0.018,-0.011,60.0
21290,110.0,79.191502,60.145427,10.024241,0.595425,18.0,18.0,99.0,0.009,0.016,-0.001,60.0
1385,159.0,71.172095,53.128466,15.036361,0.535129,18.0,18.0,99.0,-0.040,-0.007,-0.011,50.0
11429,100.0,73.176941,53.128466,14.033932,0.550203,18.0,18.0,97.0,-0.006,0.010,-0.003,40.0
29980,111.0,98.237518,65.157550,14.033932,0.738628,18.0,18.0,98.0,-0.020,-0.003,-0.002,40.0


In [20]:
len(df[df.Tag==1])

8272

In [61]:
#Add column:

data['HR_AVG'] = pd.Series('NULL', index=data.index)
data['ArtBPS_AVG'] = pd.Series('NULL', index=data.index)
data['ArtBPM_AVG'] = pd.Series('NULL', index=data.index)
data['CVP_AVG'] = pd.Series('NULL', index=data.index)
data['PAPD_AVG'] = pd.Series('NULL', index=data.index)
data['RR_total_AVG'] = pd.Series('NULL', index=data.index)
data['RR_mandatory_AVG'] = pd.Series('NULL', index=data.index)
data['Spo2_AVG'] = pd.Series('NULL', index=data.index)
data['ST1_AVG'] = pd.Series('NULL', index=data.index)
data['ST2_AVG'] = pd.Series('NULL', index=data.index)
data['ST3_AVG'] = pd.Series('NULL', index=data.index)
data['ST2_AVG'] = pd.Series('NULL', index=data.index)
data['Fio2_AVG'] = pd.Series('NULL', index=data.index)

In [94]:
avg=data[3:5].mean()
avg

HR              105.500000
ArtBPS           85.707229
ArtBPM           59.143008
CVP              14.033932
PAPD              0.644415
RR_total         18.000000
RR_mandatory     18.000000
Spo2             97.500000
ST1              -0.013000
ST2               0.003500
ST3              -0.002500
Fio2             40.000000
dtype: float64

In [96]:
for i in range(4,len(data.index)):
    p1=df.ix[i-1,'PatientID']
    p3=df.ix[i-3,'PatientID']
    t1=df.ix[i-1,'Time'].to_pydatetime()
    t3=df.ix[i-3,'Time'].to_pydatetime()
    if (t3-t1).total_seconds()<=600.0 and p1==p3:
        avg=data[i-3:i-1].mean()
        for j in range(1,12):
            data[i,j+13]=avg[j]

KeyboardInterrupt: 

In [ ]:
df.to_csv(r'C:\Users\TEMP\Desktop\Python - Thesis\df.csv', header= 'TRUE', index=None, sep=',', mode='a')